In [1]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'NanumGothicCoding'

In [2]:
import pandas as pd
import numpy as np

In [156]:
df_cabbage = pd.read_csv("../data/2006to2015_chinise_cabbage.csv", encoding='cp949')
df_radish = pd.read_csv("../data/2006to2015_radish.csv", encoding='cp949')
df_pepper = pd.read_csv("../data/2006to2015_red_pepper.csv", encoding='cp949')

df_weather = pd.read_csv("../data/2006to2015_weather.csv", encoding='cp949')

In [199]:
# 원본 데이터를 다시 불러와야 할 경우를 대비해 복사해서 사용
df_cabbage_copied = df_cabbage.copy()
df_radish_copied = df_radish.copy()
df_pepper_copied = df_pepper.copy()

In [200]:
# 컬럼 삭제 및 변경
def column_drop_and_change(df, species):
    if species == 0:
        change_columns = {'.1': '노지봄배추', '.3': '고랭지배추', '.5': '노지가을배추'}
        drop_columns = ['.2', '.4', '.6', '.7']
        drop_year_columns = list(map(str, np.arange(2006, 2016)))
        total_drop_columns = drop_year_columns.copy()
        
        # drop columns 구하기
        for dyc in drop_year_columns:
             for drop_col in drop_columns:
                    for col in df.columns:
                        if col == dyc+drop_col:
                            total_drop_columns.append(col)

        columns_to_change = df_cabbage_copied.drop(total_drop_columns, axis=1).columns
        df_cabbage_copied.drop(total_drop_columns, axis=1, inplace=True)

        for col_to_change in columns_to_change:
            for change_col in change_columns:
                if col_to_change[-2:] == change_col:
                    df.loc[0, col_to_change] = change_columns[change_col]
                    df.rename(columns={col_to_change:col_to_change[:-2]}, inplace=True)
                    
    elif species == 1:
        change_columns = {'.1': '노지봄무', '.3': '고랭지무', '.5': '노지가을무', '.7': '노지겨울무'}
        drop_columns = ['.2', '.4', '.6']
        drop_year_columns = list(map(str, np.arange(2006, 2016)))
        total_drop_columns = drop_year_columns.copy()
        
    
        # drop columns 구하기
        for dyc in drop_year_columns:
             for drop_col in drop_columns:
                    for col in df.columns:
                        if col == dyc+drop_col:
                            total_drop_columns.append(col)

        columns_to_change = df_radish_copied.drop(total_drop_columns, axis=1).columns
        df_radish_copied.drop(total_drop_columns, axis=1, inplace=True)

        for col_to_change in columns_to_change:
            for change_col in change_columns:
                if col_to_change[-2:] == change_col:
                    df.loc[0, col_to_change] = change_columns[change_col]
                    df.rename(columns={col_to_change:col_to_change[:-2]}, inplace=True)
                    
    elif species == 2:
        change_columns = {'.1': '홍고추'}
        drop_year_columns = list(map(str, np.arange(2006, 2016)))
        total_drop_columns = drop_year_columns.copy()
        
        columns_to_change = df_pepper_copied.drop(total_drop_columns, axis=1).columns
        df_pepper_copied.drop(total_drop_columns, axis=1, inplace=True)
        
        for col_to_change in columns_to_change:
            df.loc[0, col_to_change] = change_columns.get('.1')
            df.rename(columns={col_to_change:col_to_change[:-2]}, inplace=True)
        
    return df

In [201]:
df_cabbage_change_columns = column_drop_and_change(df_cabbage_copied, 0)
df_radish_change_columns = column_drop_and_change(df_radish_copied, 1)
df_pepper_change_columns = column_drop_and_change(df_pepper_copied, 2)

In [203]:
# 인덱스로 지정할 컬럼도 바꿔주기
df_cabbage_change_columns.loc[0, '시도별'] = '품종'
df_cabbage_change_columns.rename(columns={'시도별':'연도별'}, inplace=True)
df_radish_change_columns.loc[0, '시도별'] = '품종'
df_radish_change_columns.rename(columns={'시도별':'연도별'}, inplace=True)
df_pepper_change_columns.loc[0, '주산지'] = '품종'
df_pepper_change_columns.rename(columns={'주산지':'연도별'}, inplace=True)

In [205]:
df_cabbage_transpose = df_cabbage_change_columns.set_index('연도별').T
df_radish_transpose = df_radish_change_columns.set_index('연도별').T
df_pepper_transpose = df_pepper_change_columns.set_index('연도별').T

In [212]:
# 자료형 변환
def type_conversion(df):
    df = df.replace('-', np.NaN)
    for col in df.columns[1:]:
        df[col] = df[col].astype(float)
    
    return df

In [213]:
df_cabbage_transpose = type_conversion(df_cabbage_transpose)
df_radish_transpose = type_conversion(df_radish_transpose)
df_pepper_transpose = type_conversion(df_pepper_transpose)

In [224]:
# 새로운 데이터 프레임 추출
df_cabbage_new = pd.DataFrame(columns=['생산 연도', '품종', '10a당 생산량', '지역'])
df_radish_new = pd.DataFrame(columns=['생산 연도', '품종', '10a당 생산량', '지역'])
df_pepper_new = pd.DataFrame(columns=['생산 연도', '품종', '10a당 생산량', '지역'])

In [270]:
def create_new_df(df_new, species):
    if species == 0:
        idx = 0
        spring_locations = ['강원도', '충청남도', '전라남도', '전라북도', '경상북도']
        highlands_locations = ['강원도', '전라북도', '경상북도', '경상남도']
        autumn_locations = ['충청남도', '충청북도', '전라남도', '전라북도']

        for row_num in range(len(df_cabbage_transpose.index)):
            if df_cabbage_transpose.iloc[row_num]['품종'] == '노지봄배추':
                for locate in spring_locations:
                    df_new.loc[idx] = [df_cabbage_transpose.index[row_num], df_cabbage_transpose.iloc[row_num]['품종'], df_cabbage_transpose.iloc[row_num][locate], locate]
                    idx += 1
            if df_cabbage_transpose.iloc[row_num]['품종'] == '고랭지배추':
                for locate in highlands_locations:
                    df_new.loc[idx] = [df_cabbage_transpose.index[row_num], df_cabbage_transpose.iloc[row_num]['품종'], df_cabbage_transpose.iloc[row_num][locate], locate]
                    idx += 1
            if df_cabbage_transpose.iloc[row_num]['품종'] == '노지가을배추':
                for locate in autumn_locations:
                    df_new.loc[idx] = [df_cabbage_transpose.index[row_num], df_cabbage_transpose.iloc[row_num]['품종'], df_cabbage_transpose.iloc[row_num][locate], locate]
                    idx += 1
    elif species == 1:
        idx = 0
        spring_locations = ['경기도', '강원도', '충청남도', '전라남도', '경상남도']
        highlands_locations = ['강원도', '충청북도', '전라북도', '경상북도']
        autumn_locations = ['경기도', '강원도', '충청남도', '전라북도', '경상북도']
        winter_locations = ['제주도']

        for row_num in range(len(df_radish_transpose.index)):
            if df_radish_transpose.iloc[row_num]['품종'] == '노지봄무':
                for locate in spring_locations:
                    df_new.loc[idx] = [df_radish_transpose.index[row_num], df_radish_transpose.iloc[row_num]['품종'], df_radish_transpose.iloc[row_num][locate], locate]
                    idx += 1
            if df_radish_transpose.iloc[row_num]['품종'] == '고랭지무':
                for locate in highlands_locations:
                    df_new.loc[idx] = [df_radish_transpose.index[row_num], df_radish_transpose.iloc[row_num]['품종'], df_radish_transpose.iloc[row_num][locate], locate]
                    idx += 1
            if df_radish_transpose.iloc[row_num]['품종'] == '노지가을무':
                for locate in autumn_locations:
                    df_new.loc[idx] = [df_radish_transpose.index[row_num], df_radish_transpose.iloc[row_num]['품종'], df_radish_transpose.iloc[row_num][locate], locate]
                    idx += 1
            if df_radish_transpose.iloc[row_num]['품종'] == '노지겨울무':
                for locate in winter_locations:
                    df_new.loc[idx] = [df_radish_transpose.index[row_num], df_radish_transpose.iloc[row_num]['품종'], df_radish_transpose.iloc[row_num][locate], locate]
                    idx += 1
                    
    elif species == 2:
        idx = 0
        locations = ['충북 제천', '전북 고창', '전남 해남', '전남 영광', '경북 안동', '경북 의성', '경북 청송', '경북 영주', '경북 봉화']

        for row_num in range(len(df_pepper_transpose.index)):
            for locate in locations:
                locate_name = locate[-2:]
                if locate in ['전남 영광', '경북 청송']:
                    locate_name += '군'
                elif locate == '전북 고창':
                    df_new.loc[idx] = [df_pepper_transpose.index[row_num], df_pepper_transpose.iloc[row_num]['품종'], df_pepper_transpose.iloc[row_num][locate], locate_name+'군']
                    idx += 1
                df_new.loc[idx] = [df_pepper_transpose.index[row_num], df_pepper_transpose.iloc[row_num]['품종'], df_pepper_transpose.iloc[row_num][locate], locate_name]
                idx += 1
    return df_new

In [271]:
df_cabbage_new = create_new_df(df_cabbage_new, 0)
df_radish_new = create_new_df(df_radish_new, 1)
df_pepper_new = create_new_df(df_pepper_new, 2)

In [272]:
df_cabbage_new

,생산 연도,품종,10a당 생산량,지역
0,2006,노지봄배추,4068.0,강원도
1,2006,노지봄배추,4428.0,충청남도
2,2006,노지봄배추,5781.0,전라남도
3,2006,노지봄배추,4462.0,전라북도
4,2006,노지봄배추,4607.0,경상북도
...,...,...,...,...
125,2015,고랭지배추,5317.0,경상남도
126,2015,노지가을배추,11161.0,충청남도
127,2015,노지가을배추,10961.0,충청북도
128,2015,노지가을배추,12265.0,전라남도


In [273]:
df_radish_new

,생산 연도,품종,10a당 생산량,지역
0,2006,노지봄무,3478.0,경기도
1,2006,노지봄무,3058.0,강원도
2,2006,노지봄무,3665.0,충청남도
3,2006,노지봄무,4370.0,전라남도
4,2006,노지봄무,3980.0,경상남도
...,...,...,...,...
140,2015,노지가을무,7153.0,강원도
141,2015,노지가을무,9360.0,충청남도
142,2015,노지가을무,10242.0,전라북도
143,2015,노지가을무,7574.0,경상북도


In [274]:
df_pepper_new

,생산 연도,품종,10a당 생산량,지역
0,2006,홍고추,247.0,제천
1,2006,홍고추,212.0,고창군
2,2006,홍고추,212.0,고창
3,2006,홍고추,197.0,해남
4,2006,홍고추,242.0,영광군
...,...,...,...,...
95,2015,홍고추,299.0,안동
96,2015,홍고추,304.0,의성
97,2015,홍고추,305.0,청송군
98,2015,홍고추,299.0,영주


In [230]:
df_weather.head()

,지점,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),평균상대습도(%),월합강수량(00~24h만)(mm),평균풍속(m/s),합계 일사량(MJ/m2)
0,90,속초,2006-01,0.2,8.3,-9.6,54.0,37.1,2.4,NaN
1,90,속초,2006-02,1.5,16.1,-11.9,49.0,18.3,2.9,NaN
2,90,속초,2006-03,6.0,18.9,-6.4,45.0,9.3,3.2,NaN
3,90,속초,2006-04,9.5,25.6,1.9,63.0,90.2,3.1,NaN
4,90,속초,2006-05,14.4,29.8,3.8,72.0,141.7,2.4,NaN


In [231]:
df_weather['지점명'].unique()

array(['속초', '철원', '동두천', '파주', '대관령', '춘천', '백령도', '북강릉', '강릉', '동해',
       '서울', '인천', '원주', '울릉도', '수원', '영월', '충주', '서산', '울진', '청주', '대전',
       '추풍령', '안동', '상주', '포항', '군산', '대구', '전주', '울산', '창원', '광주', '부산',
       '통영', '목포', '여수', '흑산도', '완도', '고창', '순천', '진도(첨찰산)', '대구(기)',
       '홍성', '제주', '고산', '성산', '서귀포', '진주', '강화', '양평', '이천', '인제', '홍천',
       '태백', '정선군', '제천', '보은', '천안', '보령', '부여', '금산', '부안', '임실', '정읍',
       '남원', '장수', '고창군', '영광군', '김해시', '순창군', '북창원', '주암', '양산시', '보성군',
       '강진군', '장흥', '해남', '고흥', '의령군', '함양군', '성산포', '광양시', '진도군', '봉화',
       '영주', '문경', '청송군', '영덕', '의성', '구미', '영천', '경주시', '거창', '합천', '밀양',
       '산청', '거제', '남해'], dtype=object)

In [232]:
df_weather_copied = df_weather.copy()

In [233]:
def weather_filter(df):
    location = {'강원도' : ['강릉', '대관령', '동해', '북강릉', '북춘천', '삼척', '속초', '영월', '원주', '인제', '정선군', '철원', '춘천', '태백', '홍천'],
                '경기도' : ['동두천', '수원', '양평', '이천', '파주'],
                '경상남도' : ['거제', '거창', '김해시', '남해', '밀양', '북창원', '산청', '양산시', '의령군', '진주', '창원', '통영', '함양군', '합천'],
                '경상북도' : ['경주시', '구미', '문경', '봉화', '상주', '안동', '영덕', '영주', '영천', '울릉도', '울진', '의성', '청송군', '포항'],
                '광주광역시' : ['광주'],
                '대구광역시' : ['대구', '대구(기)'],
                '대전광역시' : ['대전'],
                '부산광역시' : ['부산'],
                '서울특별시' : ['관악산', '서울'],
                '세종특별자치시' : ['세종'],
                '울산광역시' : ['울산'],
                '인천광역시' : ['강화', '백령도', '인천'],
                '전라남도' : ['강진군', '고흥', '광양시', '목포', '무안', '보성군', '순천', '여수', '영광군', '완도', '장흥', '주암', '진도(첨찰산)', '진도군', '해남', '흑산도'],
                '전라북도' : ['고창', '고창군', '군산', '남원', '부안', '순창군', '임실', '장수', '전주', '정읍'],
                '제주도' : ['고산', '서귀포', '성산', '성산포', '제주'],
                '충청남도' : ['금산', '보령', '부여', '서산', '천안', '홍성'],
                '충청북도' : ['보은', '제천', '청주', '추풍령', '충주']}
    
    idx = 0
    trans_location = []
    idx_numbers = []
    for row_num in range(len(df.index)):
        trans_location.append([k for k, v in location.items() if df.iloc[row_num]['지점명'] in v][0])
        
    df['지점명'] = trans_location
    
    for row_num in range(len(df.index)):
        if df.iloc[row_num]['일시'][-2:] in ['01', '11', '12']:
            idx_numbers.append(row_num)
    df = df.drop(idx_numbers)
    return df

In [234]:
df_weather_trans = weather_filter(df_weather_copied)

In [235]:
df_weather_trans

,지점,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),평균상대습도(%),월합강수량(00~24h만)(mm),평균풍속(m/s),합계 일사량(MJ/m2)
1,90,강원도,2006-02,1.5,16.1,-11.9,49.0,18.3,2.9,NaN
2,90,강원도,2006-03,6.0,18.9,-6.4,45.0,9.3,3.2,NaN
3,90,강원도,2006-04,9.5,25.6,1.9,63.0,90.2,3.1,NaN
4,90,강원도,2006-05,14.4,29.8,3.8,72.0,141.7,2.4,NaN
5,90,강원도,2006-06,18.4,30.8,12.1,79.0,129.0,2.3,NaN
...,...,...,...,...,...,...,...,...,...,...
10455,295,경상남도,2015-06,21.2,30.6,13.7,74.0,101.6,1.1,NaN
10456,295,경상남도,2015-07,24.4,34.1,16.6,78.0,343.7,1.3,NaN
10457,295,경상남도,2015-08,25.6,34.7,18.3,76.0,166.0,1.1,NaN
10458,295,경상남도,2015-09,21.4,29.8,14.5,73.0,120.1,1.3,NaN


In [250]:
df_weather_cabbage = pd.DataFrame(columns=['평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '평균상대습도(%)', '월합강수량(mm)', '평균풍속(m/s)', '합계 일사량(MJ/m2)'])
df_weather_radish = pd.DataFrame(columns=['평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '평균상대습도(%)', '월합강수량(mm)', '평균풍속(m/s)', '합계 일사량(MJ/m2)'])
df_weather_pepper = pd.DataFrame(columns=['평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '평균상대습도(%)', '월합강수량(mm)', '평균풍속(m/s)', '합계 일사량(MJ/m2)'])

In [282]:
def merge_df(df_new, df_crop, df_climate, species):
    if species == 0:
        for row_num in range(len(df_crop.index)):
            if df_crop.iloc[row_num]['품종'] == '노지봄배추':
                idx = ((df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'03') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'04') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'05')) & (df_climate.지점명 == df_crop.iloc[row_num]['지역'])
            elif df_crop.iloc[row_num]['품종'] == '고랭지배추':
                idx = ((df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'05') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'06') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'07')) & (df_climate.지점명 == df_crop.iloc[row_num]['지역'])
            elif df_crop.iloc[row_num]['품종'] == '노지가을배추':
                idx = ((df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'08') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'09') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'10')) & (df_climate.지점명 == df_crop.iloc[row_num]['지역'])
            df_new.loc[row_num] = [df_climate[idx].mean()[1], df_climate[idx].mean()[2], df_climate[idx].mean()[3], df_climate[idx].mean()[4], df_climate[idx].mean()[5], df_climate[idx].mean()[6], df_climate[idx].mean()[7]]

    elif species == 1:
        for row_num in range(len(df_crop.index)):
            if df_crop.iloc[row_num]['품종'] == '노지봄무':
                idx = ((df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'03') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'04') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'05')) & (df_climate.지점명 == df_crop.iloc[row_num]['지역'])
            elif df_crop.iloc[row_num]['품종'] == '고랭지무':
                idx = ((df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'05') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'06') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'07')) & (df_climate.지점명 == df_crop.iloc[row_num]['지역'])
            elif df_crop.iloc[row_num]['품종'] == '노지가을무':
                idx = ((df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'07') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'08') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'09')) & (df_climate.지점명 == df_crop.iloc[row_num]['지역'])
            elif df_crop.iloc[row_num]['품종'] == '노지겨울무':
                idx = ((df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'09') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'10') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'11')) & (df_climate.지점명 == df_crop.iloc[row_num]['지역'])
            df_new.loc[row_num] = [df_climate[idx].mean()[1], df_climate[idx].mean()[2], df_climate[idx].mean()[3], df_climate[idx].mean()[4], df_climate[idx].mean()[5], df_climate[idx].mean()[6], df_climate[idx].mean()[7]]
    
    elif species == 2:
        for row_num in range(len(df_crop.index)):
            idx = ((df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'02') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'03') | (df_climate.일시 == str(df_crop.iloc[row_num]['생산 연도'])+'-'+'04')) & (df_climate.지점명 == df_crop.iloc[row_num]['지역'])
            df_new.loc[row_num] = [df_climate[idx].mean()[1], df_climate[idx].mean()[2], df_climate[idx].mean()[3], df_climate[idx].mean()[4], df_climate[idx].mean()[5], df_climate[idx].mean()[6], df_climate[idx].mean()[7]]

    return df_new

In [283]:
merge_df(df_weather_cabbage, df_cabbage_new, df_weather_trans, 0)
merge_df(df_weather_radish, df_radish_new, df_weather_trans, 1)
merge_df(df_weather_pepper, df_pepper_new, df_weather, 2)
print('finish')

/tmp/ipykernel_21/1912275782.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_new.loc[row_num] = [df_climate[idx].mean()[1], df_climate[idx].mean()[2], df_climate[idx].mean()[3], df_climate[idx].mean()[4], df_climate[idx].mean()[5], df_climate[idx].mean()[6], df_climate[idx].mean()[7]]
/tmp/ipykernel_21/1912275782.py:22: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_new.loc[row_num] = [df_climate[idx].mean()[1], df_climate[idx].mean()[2], df_climate[idx].mean()[3], df_climate[idx].mean()[4], df_climate[idx].mean()[5], df_climate[idx].mean()[6], df_climate[idx].mean()[7]]
/tmp/ipykernel_21/1912275782.py:27: FutureWarning: Dropping of nuisance 

finish


In [305]:
df_weather_cabbage.sort_index(inplace=True)

In [306]:
df_weather_radish.sort_index(inplace=True)

In [307]:
df_weather_pepper.sort_index(inplace=True)

In [314]:
final_cabbage = pd.concat([df_cabbage_new,df_weather_cabbage],axis=1)
final_radish = pd.concat([df_radish_new,df_weather_radish],axis=1)
final_pepper = pd.concat([df_pepper_new,df_weather_pepper],axis=1)

In [315]:
final_cabbage

,생산 연도,품종,10a당 생산량,지역,평균기온(°C),최고기온(°C),최저기온(°C),평균상대습도(%),월합강수량(mm),평균풍속(m/s),합계 일사량(MJ/m2)
0,2006,노지봄배추,4068.0,강원도,9.887879,23.148485,-2.309091,58.545455,77.875758,2.518182,495.906667
1,2006,노지봄배추,4428.0,충청남도,10.760000,23.420000,-0.666667,63.733333,64.553333,2.206667,512.423333
2,2006,노지봄배추,5781.0,전라남도,11.233333,23.311111,1.137037,70.629630,136.366667,3.855556,511.088333
3,2006,노지봄배추,4462.0,전라북도,11.219048,25.166667,-0.285714,65.904762,83.585714,2.080952,479.383333
4,2006,노지봄배추,4607.0,경상북도,11.175000,25.436111,-0.991667,60.555556,86.783333,2.588889,439.190000
...,...,...,...,...,...,...,...,...,...,...,...
125,2015,고랭지배추,5317.0,경상남도,21.476190,33.076190,11.130952,71.476190,143.873810,1.616667,560.119444
126,2015,노지가을배추,11161.0,충청남도,20.013333,30.466667,9.226667,78.600000,55.040000,1.386667,363.680000
127,2015,노지가을배추,10961.0,충청북도,19.153333,30.400000,7.620000,70.066667,60.026667,1.426667,463.296667
128,2015,노지가을배추,12265.0,전라남도,20.638095,29.747619,12.104762,79.095238,100.145238,2.159524,470.223333


In [316]:
final_radish

,생산 연도,품종,10a당 생산량,지역,평균기온(°C),최고기온(°C),최저기온(°C),평균상대습도(%),월합강수량(mm),평균풍속(m/s),합계 일사량(MJ/m2)
0,2006,노지봄무,3478.0,경기도,11.140000,23.653333,-0.286667,59.266667,72.346667,2.053333,503.326667
1,2006,노지봄무,3058.0,강원도,9.887879,23.148485,-2.309091,58.545455,77.875758,2.518182,495.906667
2,2006,노지봄무,3665.0,충청남도,10.760000,23.420000,-0.666667,63.733333,64.553333,2.206667,512.423333
3,2006,노지봄무,4370.0,전라남도,11.233333,23.311111,1.137037,70.629630,136.366667,3.855556,511.088333
4,2006,노지봄무,3980.0,경상남도,12.400000,24.962963,0.811111,62.814815,132.555556,2.022222,508.333333
...,...,...,...,...,...,...,...,...,...,...,...
140,2015,노지가을무,7153.0,강원도,22.056410,32.782051,12.984615,76.769231,124.133333,1.705128,494.448000
141,2015,노지가을무,9360.0,충청남도,23.333333,32.873333,14.040000,81.333333,71.213333,1.533333,400.830000
142,2015,노지가을무,10242.0,전라북도,23.276667,33.406667,14.536667,78.966667,91.400000,1.636667,537.524167
143,2015,노지가을무,7574.0,경상북도,22.616667,33.838095,13.747619,76.904762,100.433333,1.797619,474.045000


In [317]:
final_pepper.drop('합계 일사량(MJ/m2)', axis=1, inplace=True)

In [318]:
final_pepper.dropna(inplace=True)
final_pepper.reset_index(drop=True, inplace=True)

In [319]:
final_pepper

,생산 연도,품종,10a당 생산량,지역,평균기온(°C),최고기온(°C),최저기온(°C),평균상대습도(%),월합강수량(mm),평균풍속(m/s)
0,2006,홍고추,247.0,제천,4.300000,18.033333,-10.066667,58.000000,50.466667,1.933333
1,2006,홍고추,197.0,해남,6.500000,18.900000,-5.466667,70.000000,63.500000,3.033333
2,2006,홍고추,276.0,안동,5.600000,20.500000,-7.800000,63.000000,46.866667,2.266667
3,2006,홍고추,196.0,의성,5.200000,21.500000,-9.833333,59.666667,38.333333,1.533333
4,2006,홍고추,274.0,영주,5.466667,19.966667,-6.966667,57.000000,60.833333,3.333333
...,...,...,...,...,...,...,...,...,...,...
81,2015,홍고추,299.0,안동,6.800000,21.266667,-6.333333,54.666667,41.166667,2.000000
82,2015,홍고추,304.0,의성,5.766667,21.866667,-8.666667,60.333333,38.466667,1.533333
83,2015,홍고추,305.0,청송군,5.566667,21.733333,-8.266667,57.333333,42.600000,1.666667
84,2015,홍고추,299.0,영주,6.466667,20.400000,-6.766667,54.666667,50.366667,3.166667


In [321]:
final_cabbage.to_csv('final_cabbage.csv')
final_radish.to_csv('final_radish.csv')
final_pepper.to_csv('final_pepper.csv')